In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
%cd '/content/drive/Shareddrives/SWC_Capstone/5월'

/content/drive/Shareddrives/SWC_Capstone/5월


In [18]:
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
import numpy as np
import pandas as pd

In [19]:
!pip install -q wordcloud

# 텍스트 토큰화

In [ ]:
depression = pd.read_csv("./psy_data/year_month_col/depress_ym.csv")
bipolar = pd.read_csv("./psy_data/year_month_col/bipolar_ym.csv")
panic = pd.read_csv("./psy_data/year_month_col/panic_ym.csv")
covid19 = pd.read_csv("./psy_data/year_month_col/covid_ym.csv")
relationship = pd.read_csv("./psy_data/year_month_col/relationship_ym.csv")
teaching = pd.read_csv("./psy_data/year_month_col/teaching_ym.csv")

In [5]:
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))
stop_words.extend(["'m"])
print()
new_stopwords = []
temp = ['hadn','mightn','mustn','wasn','couldn','doesn','hasn','ain','shan','aren','weren','wouldn','shouldn']
for word in stop_words:
    if 'not' in word: continue
    elif "'t" in word: continue
    elif len(word) == 1: continue
    elif word in temp: continue
    elif '!' in word: continue
    elif '.' in word: continue
    else: new_stopwords.append(word)
        
# 감정분석용 cleansing
def s_cleansing(List):
    count=0
    corpus = []
    for i in List:
        i = str(i)# 정규화 에러로 인해 추가
        url_pattern ='https?://\S+|#([0-9a-zA-Z]*)'
        i = re.sub(pattern=url_pattern, repl=' ', string = i)
        i = i.replace('\r','').replace('\n','')
        text = re.sub('[-=+,#/\?:^$”@*\"“※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》;’]', '', i.lower())
        corpus.append(text)
        count+=1
        #if(count>24900): print(count)        
    return corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
dep_mix = []
for i in (depression['title'] + ' '+depression['text_context']):
    dep_mix.append(i)
    
bipolar_mix = []
for i in (bipolar['title'] + ' '+bipolar['text_context']):
    bipolar_mix.append(i)
    
panic_mix = []
for i in (panic['title'] + ' '+panic['text_context']):
    panic_mix.append(i)

covid19_mix = []
for i in (covid19['title'] + ' '+covid19['text_context']):
    covid19_mix.append(i)
    
relationship_mix = []
for i in (relationship['title'] + ' '+relationship['text_context']):
    relationship_mix.append(i)
    
teaching_mix = []
for i in (teaching['title'] + ' '+teaching['text_context']):
    teaching_mix.append(i)

In [ ]:
dep_cleansing = s_cleansing(dep_mix)
bipolar_cleansing = s_cleansing(bipolar_mix)
anxiety_cleansing = s_cleansing(panic_mix)

covid19_cleansing = s_cleansing(covid19_mix)
teaching_cleansing = s_cleansing(teaching_mix)
relationship_cleansing = s_cleansing(relationship_mix)


# Valence-Arousal Scoring Algorithn

## 사전 다운로드

In [8]:
lexicon_Val = pd.read_csv("./NRC-VAD-Lexicon-Aug2018Release/OneFilePerDimension/v-scores.txt", sep = '\t')
lexicon_Val.columns =['Emotion','Score']
lexicon_Val=lexicon_Val.dropna()
lexicon_Val = lexicon_Val.reset_index()
lexicon_Val = lexicon_Val.drop(columns = ['index'])

lexicon_Aro = pd.read_csv("./NRC-VAD-Lexicon-Aug2018Release/OneFilePerDimension/a-scores.txt", sep = '\t')
lexicon_Aro.columns =['Emotion','Score']
lexicon_Aro=lexicon_Aro.dropna()
lexicon_Aro = lexicon_Aro.reset_index()
lexicon_Aro = lexicon_Aro.drop(columns = ['index'])


## 기존 알고리즘

In [ ]:
# V,A 중 하나의 데이터셋만 다루는 함수 

# selected : tokenize by word original
# https://github.com/Priya22/EmotionDynamics

# 이후 VADER의 부정어 처리에 맞는 tokenizer 선택
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def score_by_lex_original(lexicon, text_df, save_path):#lexicon scoring algorithm
    numTokens = []
    numLexTokens = []
    avgLexVal = []
    checkpoint = 0
    for t in text_df:
        score = []
        text = str(t)
        tokenized = text_to_word_sequence(text)
        numTokens.append(len(tokenized))

        for token in tokenized:
            index = 0
            for emotion in lexicon['Emotion']:
                if emotion in token:
                    score.append(lexicon['Score'][index])
                    break                
                index += 1
                
        checkpoint += 1
        if checkpoint % 1000 == 0: # 중간 저장 과정
            print(check)  
            result_DF = pd.DataFrame((zip(numTokens, numLexTokens, avgLexVal)), columns = ['numTokens', 'numLexTokens', 'avgLexVal'])
            result_DF.to_csv(save_path, mode='w', index = False)
        
        numLexTokens.append(len(score))
        
        if len(score) != 0: average_score = sum(score) / len(score)
        else: average_score = 0.5
        avgLexVal.append(average_score)
            
    # 파일 저장 확인
    result_DF = pd.DataFrame((zip(numTokens, numLexTokens, avgLexVal)), columns = ['numTokens', 'numLexTokens', 'avgLexVal'])
    result_DF.to_csv(save_path, mode='w', index = False)
    
    return result_DF l   

# 1000개 게시글 처리하는데 걸리는 시간 약 10-12분

In [ ]:
## 점수 매기는 과정 (without non mental health)

dep_val_score = score_by_lex_original(lexicon_Val, dep_cleansing,'./scoring_result/depression_valence_score.csv')
dep_aro_score = score_by_lex_original(lexicon_Aro, dep_cleansing,'./scoring_result/depression_arousal_score.csv')

bipolar_val_score = score_by_lex_original(lexicon_Val, bipolar_cleansing,'./scoring_result/depression_valence_score.csv')
bipolar_aro_score = score_by_lex_original(lexicon_Aro, bipolar_cleansing,'./scoring_result/depression_arousal_score.csv')

panic_val_score = score_by_lex_original(lexicon_Val, anxiety_cleansing,'./scoring_result/depression_valence_score.csv')
panic_aro_score = score_by_lex_original(lexicon_Aro, anxiety_cleansing,'./scoring_result/depression_arousal_score.csv')


In [ ]:
## 결과 출력
print("DEPRESSION")
dep_val_score = pd.read_csv("./scoring_result/depression_valence_score.csv")
print("V : ",sum(dep_val_score)/len(dep_val_score))
print("A : ",sum(dep_aro_score)/len(dep_aro_score))

print("BIPOLAR")
bipolar_val_score = pd.read_csv("./scoring_result/bipolar_valence_score.csv")
print("V : ",sum(bipolar_val_score)/len(bipolar_val_score))
print("A : ",sum(bipolar_aro_score)/len(bipolar_aro_score))

print("PANIC")
panic_val_score = pd.read_csv("./scoring_result/panic_valence_score.csv")
print("V : ",sum(panic_val_score)/len(panic_val_score))
print("A : ",sum(panic_aro_score)/len(panic_aro_score))

## 수정한 알고리즘

### 데이터셋 구성 변경> 사전 전처리

In [35]:
B_INCR = 0.293
B_DECR = -0.293

C_INCR = 0.733
N_SCALAR = -0.74

NEGATE = \
    ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
     "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
     "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
     "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
     "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
     "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
     "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
     "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

# http://en.wiktionary.org/wiki/Category:English_degree_adverbs

BOOSTER_DICT = \
    {"absolutely": B_INCR, "amazingly": B_INCR, "awfully": B_INCR,
     "completely": B_INCR, "considerable": B_INCR, "considerably": B_INCR,
     "decidedly": B_INCR, "deeply": B_INCR, "effing": B_INCR, "enormous": B_INCR, "enormously": B_INCR,
     "entirely": B_INCR, "especially": B_INCR, "exceptional": B_INCR, "exceptionally": B_INCR,
     "extreme": B_INCR, "extremely": B_INCR,
     "fabulously": B_INCR, "flipping": B_INCR, "flippin": B_INCR, "frackin": B_INCR, "fracking": B_INCR,
     "fricking": B_INCR, "frickin": B_INCR, "frigging": B_INCR, "friggin": B_INCR, "fully": B_INCR,
     "fuckin": B_INCR, "fucking": B_INCR, "fuggin": B_INCR, "fugging": B_INCR,
     "greatly": B_INCR, "hella": B_INCR, "highly": B_INCR, "hugely": B_INCR,
     "incredible": B_INCR, "incredibly": B_INCR, "intensely": B_INCR,
     "major": B_INCR, "majorly": B_INCR, "more": B_INCR, "most": B_INCR, "particularly": B_INCR,
     "purely": B_INCR, "quite": B_INCR, "really": B_INCR, "remarkably": B_INCR,
     "so": B_INCR, "substantially": B_INCR,
     "thoroughly": B_INCR, "total": B_INCR, "totally": B_INCR, "tremendous": B_INCR, "tremendously": B_INCR,
     "uber": B_INCR, "unbelievably": B_INCR, "unusually": B_INCR, "utter": B_INCR, "utterly": B_INCR,
     "very": B_INCR,
     "almost": B_DECR, "barely": B_DECR, "hardly": B_DECR, "just enough": B_DECR,
     "kind of": B_DECR, "kinda": B_DECR, "kindof": B_DECR, "kind-of": B_DECR,
     "less": B_DECR, "little": B_DECR, "marginal": B_DECR, "marginally": B_DECR,
     "occasional": B_DECR, "occasionally": B_DECR, "partly": B_DECR,
     "scarce": B_DECR, "scarcely": B_DECR, "slight": B_DECR, "slightly": B_DECR, "somewhat": B_DECR,
     "sort of": B_DECR, "sorta": B_DECR, "sortof": B_DECR, "sort-of": B_DECR}


In [10]:
## 불용어 정의 
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))

STOP_WORDS = set()

file = open("./stopwords.txt", "r")
while True:
    line = file.readline()
    if not line:
        break
    STOP_WORDS.add(line.strip())

file.close()
len(STOP_WORDS)

for word in stop_words:
    if word not in STOP_WORDS:
        STOP_WORDS.add(word)
print(STOP_WORDS)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'anything', 'theyre', 'down', 'until', 'sufficiently', 'myself', 'hereby', 'present', 'recent', 'sometime', 'outside', 'thou', 'whole', 'didn', "shan't", 'world', 'right', 'at', 'then', 'j', 'quite', 'both', 'rd', 'eighty', 'anywhere', 'mean', 'somewhat', 'happens', 'value', 'except', 'show', 'around', 'invention', 'whence', "wouldn't", 'ok', 'afterwards', 'elsewhere', 'beyond', 'noone', 'whod', 'seven', 'possibly', 'an', 'someone', 'cause', 'only', 'giving', 'successfully', 'according', 'ought', 'ours', 'w', ',', 'importance', 'near', "needn't", 'came', 'found', 'truly', 'been', 'asking', 'towards', 'seemed', 'neither', 'slightly', 'back', 'when', 'took', '-', 'could', 'saying', 'ending', 'theyd', ']', 'having', 'looks', 'that', 'about', 'how', 'mug', 'those', '*', 'howbeit', 'nobody', 'seeing', 'none', 'thus', 'respectively', 'whereupon', 'to', 'sorry', "couldn't", 'na', 'another', "they'll", 'through', 'followed', 'on', 'ran', 'either', 'able', 'z', 'take', 'ups', "mustn't", 'she',

In [11]:
## 부정 표현, 부사어 제거

for neg in NEGATE: 
    if neg in STOP_WORDS: STOP_WORDS.remove(neg)
for booster in BOOSTER_DICT: 
    if booster in STOP_WORDS: STOP_WORDS.remove(booster)


In [12]:
for i in range(len(lexicon_Val)):
    if lexicon_Val['Emotion'].iloc[i] in BOOSTER_DICT:
        lexicon_Val.drop(index=i, axis=0)
        lexicon_Aro.drop(index=i, axis=0)

lexicon_Val.reset_index()
lexicon_Aro.reset_index()

,index,Emotion,Score
0,0,exorcism,0.980
1,1,homicide,0.973
2,2,terrorism,0.971
3,3,aggresive,0.971
4,4,bloodbath,0.971
...,...,...,...
20000,20000,slow,0.073
20001,20001,cotton,0.071
20002,20002,mellow,0.069
20003,20003,siesta,0.046


### 알고리즘 구현

In [27]:
def booster_score_update(token, emo_score):
    if emo_score < 0:
        emo_score = emo_score - BOOSTER_DICT[token]
    else: 
        emo_score = emo_score + BOOSTER_DICT[token]
    
    return emo_score

def negate_score_update(emo_score):
    return emo_score * N_SCALAR

def word_similarity(sim_word_lst, word):
    similar_lst = []
    for w in sim_word_lst:
        similar_lst.append(len(w))
    
    return np.argmax(similar_lst)  
      

In [13]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
nltk.download('averaged_perceptron_tagger')

def score_by_lex_upgrade_together(lexicon_val,lexicon_aro, text_df, save_path):#lexicon scoring algorithm
    numTokens = []
    numTokens_without_stop = []
    numLexTokens = []
    avgLexVal = []
    avgLexAro = []
    check = 0
    for t in text_df:
        
        score_lstV = []
        score_lstA = []
        text = str(t)
        tokenized = text_to_word_sequence(text)
        numTokens.append(len(tokenized))
        
        ## 불용어 제거
        token_without_stop = []
        for token in tokenized:
            if token not in STOP_WORDS:
                token_without_stop.append(token)
        numTokens_without_stop.append(len(token_without_stop))    
        
        tok_index = 0
        for token in token_without_stop:
            emo_index = 0
            emoWord_lst = []
            tempV = []
            tempA = []
            for emotion in lexicon_val['Emotion']:
                if emotion in token:
                    emoWord_lst.append(emotion)
                    ## 부정표현 & 부사 처리 알고리즘
                    val_score = lexicon_val['Score'][emo_index]
                    aro_score = lexicon_aro['Score'][emo_index]
                    check_index = tok_index - 1
                    while check_index >= 0:
                        if token_without_stop[check_index] in NEGATE:
                            val_score = negate_score_update(val_score)
                            aro_score = negate_score_update(aro_score)
                            break
                        elif token_without_stop[check_index] in BOOSTER_DICT:
                            val_score = booster_score_update(token_without_stop[check_index], val_score)
                            aro_score = booster_score_update(token_without_stop[check_index], aro_score)
                            check_index -= 1
                            continue
                        elif nltk.pos_tag([token_without_stop[check_index]])[0][1] in {'RB','RBR','RBS'}:
                            check_index -= 1
                            continue
                        else:
                            break

                    tempV.append(val_score)
                    tempA.append(aro_score)

                emo_index += 1
            tok_index += 1 

            if len(emoWord_lst) != 0:
                score_lstV.append(tempV[word_similarity(emoWord_lst,token)])
                score_lstA.append(tempA[word_similarity(emoWord_lst,token)])   
                # print(emoWord_lst[word_similarity(emoWord_lst,token)]) 

        check += 1
        if check % 1000 == 0: # 중간 저장 과정
            print(check)  
            result_DF = pd.DataFrame((zip(numTokens, numLexTokens, avgLexVal, avgLexAro)), columns = ['numTokens', 'numLexTokens', 'avgLexVal', 'avgLexAro'])
            result_DF.to_csv(save_path, mode='w', index = False)
        
        numLexTokens.append(len(score_lstV))
        
        if len(score_lstV) != 0: 
            average_score = [sum(score_lstV) / len(score_lstV), sum(score_lstA) / len(score_lstA)]
        else: average_score = [0,0]
        avgLexVal.append(average_score[0])
        avgLexAro.append(average_score[1])

        #생성되는 score list의 element 개수 df와 맞는지 체크할 것
    
    # 파일 저장 확인
    result_DF = pd.DataFrame((zip(numTokens, numLexTokens, avgLexVal, avgLexAro)), columns = ['numTokens', 'numLexTokens', 'avgLexVal', 'avgLexAro'])
    result_DF.to_csv(save_path, mode='w', index = False)
    
    return result_DF #numTokens, numTokens_without_stop, numLexTokens, avgLexVal,avgLexAro   

# dep_score = score_by_lex_upgrade_together(lexicon_Val, lexicon_Aro, dep_cleansing,'./scoring_result/upgraded_twice/depression_score.csv')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Check by sample

In [ ]:
sample_lst = []
sample_lst.append('Im in bad mood')
sample_lst.append('Im in extremely bad mood')
sample_lst.append('Im not in extremely bad mood')

sample_lst.append('I feel so gross')
sample_lst.append('I feel gross')
sample_lst.append('I dont feel so gross')

sample_lst.append('The weather was magnificent.')
sample_lst.append('The weather was kinda magnificent.')

sample_lst.append('I am sick. I\'m not really good.')
sample_lst.append('I\'m not really really good')
sample_lst.append('I\'m not really good')

sample_lst.append('It was tremendous war')
sample_lst.append('It was sort of tremendous war')
sample_lst.append('It was not tremendous war')
sample_lst.append('It was actually tremendous war')
sample_lst

['Im in bad mood',
 'Im in extremely bad mood',
 'Im not in extremely bad mood',
 'I feel so gross',
 'I feel gross',
 'I dont feel so gross',
 'The weather was magnificent.',
 'The weather was kinda magnificent.',
 "I am sick. I'm not really good.",
 "I'm not really really good",
 "I'm not really good",
 'It was tremendous war',
 'It was sort of tremendous war',
 'It was not tremendous war',
 'It was actually tremendous war']

In [ ]:
sample_lst = s_cleansing(sample_lst)

In [ ]:
sample_score_up = score_by_lex_upgrade_together(lexicon_Val, lexicon_Aro, sample_lst,'./scoring_result/upgraded_twice/sample_score.csv')

In [ ]:
sample_score_up['text'] = sample_lst
sample_score_up


,numTokens,numLexTokens,avgLexVal,avgLexAro,text
0,4,2,-0.359324,-0.204329,im in bad mood
1,5,3,-0.128553,-0.046716,im in extremely bad mood
2,6,3,0.686795,0.645068,im not in extremely bad mood
3,4,2,-0.338900,-0.102370,i feel so gross
4,3,2,-0.192400,0.044130,i feel gross
5,5,3,-0.812459,-0.781151,i dont feel so gross
6,4,2,1.750402,2.014265,the weather was magnificent.
7,5,3,1.715332,2.013059,the weather was kinda magnificent.
8,7,3,-1.519256,-1.558787,i am sick. i'm not really good.
9,5,3,-1.385735,-1.550915,i'm not really really good


In [ ]:
sample_score_origV = score_by_lex_original(lexicon_Val, sample_lst,'./scoring_result/upgraded_twice/sample_score_OV.csv')
sample_score_origA = score_by_lex_original(lexicon_Aro, sample_lst,'./scoring_result/upgraded_twice/sample_score_OV.csv')

In [ ]:
sample_score_origA.columns = ['numTokens', 'numLexTokens', 'avgLexAro']
sample_score_orig =pd.concat([sample_score_origV, sample_score_origA[['avgLexAro']]],axis=1)
sample_score_orig['text'] = sample_lst
sample_score_orig

,numTokens,numLexTokens,avgLexVal,avgLexAro,text
0,4,2,0.558755,0.599509,im in bad mood
1,5,3,0.581166,1.036992,im in extremely bad mood
2,6,3,0.581166,1.036992,im not in extremely bad mood
3,4,2,-0.192400,0.517664,i feel so gross
4,3,2,-0.192400,0.517664,i feel gross
5,5,3,0.134491,0.438741,i dont feel so gross
6,4,2,1.750402,0.526433,the weather was magnificent.
7,5,3,1.812998,0.236077,the weather was kinda magnificent.
8,7,3,0.927377,-0.175099,i am sick. i'm not really good.
9,6,3,1.579615,-0.075715,i\m not really really good


### 데이터셋 적용

In [20]:
dep_cleansing = pd.read_csv("./cleansing_dep.csv")
bipolar_cleansing = pd.read_csv("./cleansing_bip.csv")
anxiety_cleansing = pd.read_csv("./cleansing_pan.csv")

dep_cleansing = dep_cleansing['0']
bipolar_cleansing = bipolar_cleansing['0']
anxiety_cleansing = anxiety_cleansing['0']

covid_cleansing = pd.read_csv("./cleansing_cov.csv")
# relationship_cleansing = pd.read_csv("./cleansing_rel.csv")
teaching_cleansing = pd.read_csv("./cleansing_tea.csv")

covid_cleansing = covid_cleansing['0']
# relationship_cleansing = relationship_cleansing['0']
teaching_cleansing = teaching_cleansing['0']

In [ ]:
depression_score = score_by_lex_upgrade_together(lexicon_Val, lexicon_Aro, dep_cleansing,'./scoring_result/upgraded_twice/depression_score.csv')

bipolar_score = score_by_lex_upgrade_together(lexicon_Val, lexicon_Aro, bipolar_cleansing,'./scoring_result/upgraded_twice/bipolar_score.csv')

panic_score =  score_by_lex_upgrade_together(lexicon_Val, lexicon_Aro, anxiety_cleansing,'./scoring_result/upgraded_twice/panic_score.csv')

covid_score = score_by_lex_upgrade_together(lexicon_Val, lexicon_Aro, covid_cleansing,'./scoring_result/upgraded_twice/covid_score.csv')

teaching_score = score_by_lex_upgrade_together(lexicon_Val, lexicon_Aro, teaching_cleansing,'./scoring_result/upgraded_twice/teaching_score.csv')

# Category Intensity Scoring Algorithm

## category lexicon 불러오기


In [ ]:
## category lexicon 불러오기
lexicon_Cat = pd.read_csv("./Intensity_lexicon_original.csv", sep = ',')
lexicon_Cat.columns = ['word','anger','anticipation','disgust','fear','joy','sadness','surprise','trust']
lexicon_Cat = lexicon_Cat.dropna()
lexicon_Cat = lexicon_Cat.reset_index()
lexicon_Cat = lexicon_Cat.drop(columns = ['index'])

## Reddit Text dataset 불러오기

In [23]:
dep_cleansing = pd.read_csv("./cleansing_dep.csv")
bipolar_cleansing = pd.read_csv("./cleansing_bip.csv")
anxiety_cleansing = pd.read_csv("./cleansing_pan.csv")

dep_cleansing = dep_cleansing['0']
bipolar_cleansing = bipolar_cleansing['0']
anxiety_cleansing = anxiety_cleansing['0']

In [22]:
covid_cleansing = pd.read_csv("./cleansing_cov.csv")
relationship_cleansing = pd.read_csv("./cleansing_rel.csv")
teaching_cleansing = pd.read_csv("./cleansing_tea.csv")

covid_cleansing = covid_cleansing['0']
relationship_cleansing = relationship_cleansing['0']
teaching_cleansing = teaching_cleansing['0']

In [24]:
# date 정보 포함 데이터셋

depression = pd.read_csv("./psy_data/year_month_col/depress_ym.csv")
bipolar = pd.read_csv("./psy_data/year_month_col/bipolar_ym.csv")
panic = pd.read_csv("./psy_data/year_month_col/panic_ym.csv")


In [25]:
covid = pd.read_csv("./psy_data/year_month_col/covid_ym.csv")
relationship = pd.read_csv("./psy_data/year_month_col/relationship_ym.csv")
teaching = pd.read_csv("./psy_data/year_month_col/teaching_ym.csv")

## Category Check Algorithm

In [ ]:
# 별도로 normalize 필요함
def category_scoring(lexicon, clean_lst, text_df, save_path):
    if len(clean_lst) != len(text_df):
        print("wrong dataset")
        return
    
    cat_result = text_df[['date', 'year', 'month']]
    # word','anger','anticipation','disgust','fear','joy','sadness','surprise','trust']
    score_matrix = np.zeros((len(clean_lst), 8), dtype=float)

    check = 0
    for t in clean_lst:
        text = str(t)
        text_score = np.zeros(8,dtype=float)
        
        for index in range(len(lexicon)):
            emotion = lexicon['word'][index]
            row_score = np.array(lexicon.iloc[index][1:], dtype=float)
            count = text.count(emotion)
            
            row_score *= count
            text_score += row_score
        
        score_matrix[check] = text_score
        check += 1
        if check % 3000 == 0:
            print(check)  
            temp = pd.DataFrame(score_matrix, columns=['anger','anticipation','disgust','fear','joy','sadness','surprise','trust'])
            result_DF = pd.concat([cat_result, temp], axis=1)
            result_DF.to_csv(save_path, mode='w', index = False)

        # check += 1
    

    temp = pd.DataFrame(score_matrix, columns=['anger','anticipation','disgust','fear','joy','sadness','surprise','trust'])
    result_DF = pd.concat([cat_result, temp], axis=1)
    result_DF.to_csv(save_path, mode='w', index = False)

In [ ]:
category_scoring(lexicon_Cat, dep_cleansing, depression, './scoring_result/category/depression_catScore.csv')

category_scoring(lexicon_Cat, bipolar_cleansing, bipolar, './scoring_result/category/bipolar_catScore.csv')

category_scoring(lexicon_Cat, anxiety_cleansing, panic, './scoring_result/category/panic_catScore.csv')

category_scoring(lexicon_Cat, covid_cleansing, covid, './scoring_result/category/covid_catScore.csv')

category_scoring(lexicon_Cat, teaching_cleansing, teaching, './scoring_result/category/teaching_catScore.csv')

# VA Scoring Neg/Pos Independently

In [39]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
nltk.download('averaged_perceptron_tagger')

def score_by_lex_upgrade_NPInd(lexicon_val,lexicon_aro, text_df, save_path):#lexicon scoring algorithm
    numTokens = []
    numTokens_without_stop = []
    numPVTokens = []
    numNVTokens = []
    numPATokens = []
    numNATokens = []
    avgLexPVal = []
    avgLexNVal = []
    avgLexPAro = []
    avgLexNAro = []
    check = 0
    for t in text_df:
        
        score_lstV = []
        score_lstA = []
        text = str(t)
        tokenized = text_to_word_sequence(text)
        numTokens.append(len(tokenized))
        
        ## 불용어 제거
        token_without_stop = []
        for token in tokenized:
            if token not in STOP_WORDS:
                token_without_stop.append(token)
        numTokens_without_stop.append(len(token_without_stop))    
        
        tok_index = 0
        for token in token_without_stop:
            emo_index = 0
            emoWord_lst = []
            tempV = []
            tempA = []
            for emotion in lexicon_val['Emotion']:
                if emotion in token:
                    emoWord_lst.append(emotion)
                    ## 부정표현 & 부사 처리 알고리즘
                    val_score = lexicon_val['Score'][emo_index]
                    aro_score = lexicon_aro['Score'][emo_index]
                    check_index = tok_index - 1
                    while check_index >= 0:
                        if token_without_stop[check_index] in NEGATE:
                            val_score = negate_score_update(val_score)
                            aro_score = negate_score_update(aro_score)
                            break
                        elif token_without_stop[check_index] in BOOSTER_DICT:
                            val_score = booster_score_update(token_without_stop[check_index], val_score)
                            aro_score = booster_score_update(token_without_stop[check_index], aro_score)
                            check_index -= 1
                            continue
                        elif nltk.pos_tag([token_without_stop[check_index]])[0][1] in {'RB','RBR','RBS'}:
                            check_index -= 1
                            continue
                        else:
                            break

                    tempV.append(val_score)
                    tempA.append(aro_score)
                
                    
                emo_index += 1
            tok_index += 1 

            if len(emoWord_lst) != 0:
                score_lstV.append(tempV[word_similarity(emoWord_lst,token)])
                score_lstA.append(tempA[word_similarity(emoWord_lst,token)])   

        check += 1
        if check % 1000 == 0: # 중간 저장 과정
            print(check)  
            result_DF = pd.DataFrame((zip(numTokens, numNVTokens, avgLexNVal,numPVTokens, avgLexPVal,numNATokens,avgLexNAro,numPATokens,avgLexPAro)),
                                      columns = ['numTokens', 'NVnum', 'NVscore','PVnum', 'PVscore','NAnum','NAscore','PAnum','PAscore'])           
            result_DF.to_csv(save_path, mode='w', index = False)
                
        if len(score_lstV) != 0: 
            neg_score = 0
            pos_score = 0
            countN = 0
            countP = 0 
            for i in range(len(score_lstV)):
                if score_lstV[i] < 0:
                    neg_score += score_lstV[i]
                    countN +=1
                elif score_lstV[i] > 0:
                    pos_score += score_lstV[i]
                    countP += 1
            avgLexNVal.append(neg_score)
            avgLexPVal.append(pos_score)
            numNVTokens.append(countN)
            numPVTokens.append(countP)

            neg_score = 0
            pos_score = 0
            countN = 0
            countP = 0 
            for i in range(len(score_lstA)):
                if score_lstA[i] < 0:
                    neg_score += score_lstA[i]
                    countN +=1
                elif score_lstV[i] > 0:
                    pos_score += score_lstA[i]
                    countP +=1
            avgLexNAro.append(neg_score)
            avgLexPAro.append(pos_score)
            numNATokens.append(countN)
            numPATokens.append(countP)

        else:
            avgLexNVal.append(0)
            avgLexPVal.append(0)
            avgLexNAro.append(0)
            avgLexPAro.append(0)
            numNATokens.append(0)
            numPATokens.append(0)
            numNVTokens.append(0)
            numPVTokens.append(0)
        #생성되는 score list의 element 개수 df와 맞는지 체크할 것
    
    # 파일 저장 확인
    result_DF = pd.DataFrame((zip(numTokens, numNVTokens, avgLexNVal,numPVTokens, avgLexPVal,numNATokens,avgLexNAro,numPATokens,avgLexPAro)),
                              columns = ['numTokens', 'NVnum', 'NVscore','PVnum', 'PVscore','NAnum','NAscore','PAnum','PAscore'])
    result_DF.to_csv(save_path, mode='w', index = False)
    
    return result_DF #numTokens, numTokens_without_stop, numLexTokens, avgLexVal,avgLexAro   

# dep_score = score_by_lex_upgrade_together(lexicon_Val, lexicon_Aro, dep_cleansing,'./scoring_result/upgraded_twice/depression_score.csv')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [30]:
dep_cleansing = pd.read_csv("./cleansing_dep.csv")
bipolar_cleansing = pd.read_csv("./cleansing_bip.csv")
anxiety_cleansing = pd.read_csv("./cleansing_pan.csv")

dep_cleansing = dep_cleansing['0']
bipolar_cleansing = bipolar_cleansing['0']
anxiety_cleansing = anxiety_cleansing['0']

covid_cleansing = pd.read_csv("./cleansing_cov.csv")
# relationship_cleansing = pd.read_csv("./cleansing_rel.csv")
teaching_cleansing = pd.read_csv("./cleansing_tea.csv")

covid_cleansing = covid_cleansing['0']
# relationship_cleansing = relationship_cleansing['0']
teaching_cleansing = teaching_cleansing['0']

In [41]:
panic_score =  score_by_lex_upgrade_NPInd(lexicon_Val, lexicon_Aro, anxiety_cleansing,'./scoring_result/negpos_independet/panic_score.csv')

covid_score = score_by_lex_upgrade_NPInd(lexicon_Val, lexicon_Aro, covid_cleansing,'./scoring_result/negpos_independet/covid_score.csv')

In [43]:
depression_score = score_by_lex_upgrade_NPInd(lexicon_Val, lexicon_Aro, dep_cleansing,'./scoring_result/negpos_independet/depression_score.csv')

bipolar_score = score_by_lex_upgrade_NPInd(lexicon_Val, lexicon_Aro, bipolar_cleansing,'./scoring_result/negpos_independet/bipolar_score.csv')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [44]:
teaching_score = score_by_lex_upgrade_NPInd(lexicon_Val, lexicon_Aro, teaching_cleansing,'./scoring_result/negpos_independet/teaching_score.csv')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
